In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/persnality

/content/drive/MyDrive/persnality


# 모델 불러오기 

In [4]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import plotly.express as px
import pandas as pd
import re

In [5]:
cEXT = pickle.load( open( "models/cEXT.p", "rb"))
cNEU = pickle.load( open( "models/cNEU.p", "rb"))
cAGR = pickle.load( open( "models/cAGR.p", "rb"))
cCON = pickle.load( open( "models/cCON.p", "rb"))
cOPN = pickle.load( open( "models/cOPN.p", "rb"))
vectorizer_31 = pickle.load( open( "models/vectorizer_31.p", "rb"))
vectorizer_30 = pickle.load( open( "models/vectorizer_30.p", "rb"))

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.1 when using version 1.0.2. This might lead to breaking code or invalid r

In [6]:
def predict_personality(text):
    scentences = re.split("(?<=[.!?]) +", text)
    text_vector_31 = vectorizer_31.transform(scentences)
    text_vector_30 = vectorizer_30.transform(scentences)
    #EXT = cEXT.predict(text_vector_31)
    #NEU = cNEU.predict(text_vector_30)
    #AGR = cAGR.predict(text_vector_31)
    #CON = cCON.predict(text_vector_31)
    #OPN = cOPN.predict(text_vector_31)

    #print(EXT, NEU, AGR, CON, OPN)
    # return [EXT[0], NEU[0], AGR[0], CON[0], OPN[0]]

    EXT1 = cEXT.predict_proba(text_vector_31)
    NEU1 = cNEU.predict_proba(text_vector_30)
    AGR1 = cAGR.predict_proba(text_vector_31)
    CON1 = cCON.predict_proba(text_vector_31)
    OPN1 = cOPN.predict_proba(text_vector_31)
    #print(EXT1, NEU1, AGR1, CON1, OPN1)

    return [EXT1[0][1], NEU1[0][1], AGR1[0][1], CON1[0][1], OPN1[0][1]*0.8]

# 데이터 전처리 함수 

In [7]:
# 표준어
!pip install git+https://github.com/ssut/py-hanspell.git
# 띄어쓰기
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
# 번역
!pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-uuhn7box
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-uuhn7box
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4870 sha256=797ea40744b30f5679fffb463863a30de39dd0967e751d1bec31ea2b247fc252
  Stored in directory: /tmp/pip-ephem-wheel-cache-uemc3pte/wheels/3f/a5/73/e4d2806ae141d274fdddaabf8c0ed79be9357d36bfdc99e4b4
Successfully built py-hanspell
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-dnth8icr
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-dnth8icr
     |████████████████████████████████| 511.8 MB 26 kB/s 
  Created wheel for pykospacing: filename=pykospacing-0.5-

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 1.5 MB 59.3 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 65 kB 4.3 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17414 sha256=9ae0cd3db97df35cb87f182eb7e5290e0f5fce98b382bb8a875a0e5488be620d
  Stored in directory: /root/.cache/pip/wheels/a9/25/af/715361fa79594524c89c75d293def652045f3fdce6ca398712
Successfully built googletrans


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
import time

from hanspell import spell_checker
from pykospacing import Spacing
import googletrans
from googletrans import Translator

spacing = Spacing()
translator = Translator()

In [35]:
# 한글 전처리!!

def processing(sentence):

    try:
      # 한글, 공백만 추출
      sentence = sentence.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
      # 표준어
      intrd_spell = spell_checker.check(sentence)
      sentence = intrd_spell.checked
      # 띄어쓰기
      sentence = spacing(sentence)
      # 영어번역
      sentence = translator.translate(sentence, src='ko', dest='en').text

      return sentence

    except:
      pass

# APP 실행 

In [ ]:
### oracle DB 보고 자기소개+가족소개 합치기 적은거...

USER_DETAIL['INTRODUCE'] = USET_DETAIL['USER_INTRODUCE'] + USET_DETAIL['USER_INTRO_FAMILY']

In [38]:
########################## 데이터 들어갈 부분!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! #################################
## DB에서 가족소개글 + 자기소개글 컬럼만 합쳐서 내용 넣어주기

sentence = USER_DETAIL['INTRODUCE']

In [42]:
text = processing(sentence)

1/1 [==============================] - 0s 53ms/step
hello.Gwangju is a 32 -year -old pretty person.I want to marry a good person.


## Predict

In [44]:
predictions = predict_personality(text)

df = pd.DataFrame(dict(r=predictions, theta=['외향성','신경성','우호성', '성실성', '개방성']))
fig = px.line_polar(df, r='r', theta='theta', line_close=True, range_r=(0,1.0),template= 'plotly') 


fig.update_layout(
    showlegend = False,
    polar = dict(
      radialaxis_tickfont_size = 12,
      angularaxis = dict(
        tickfont_size=18,
        rotation=90,  # start position of angular axis
        direction="counterclockwise"
      )
    ))

fig.show()